# Display a map showing road segment and corresponding SRI

## Importing libraries…

In [100]:
import pandas as pd
import os
import io
import time
from selenium import webdriver
from PIL import Image
import glob
import re
from pathlib import Path
import folium
from folium import plugins

import branca
import branca.colormap as cm
from branca.element import Template, MacroElement

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# from matplotlib import cm

# plotly packages
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.express as px

## Organize Data

### Orgnize the SRI data

In [2]:
def SRI_orgnize(time_stamp):

    if os.path.exists(Path('map/SRI_' + str(time_stamp)[:10] + '.csv')): #check if SRI fila already exist in the SRI folder
        globals()[time_stamp+'SRI_df'] = pd.read_csv(Path('map/SRI_' + str(time_stamp)[:10] + '.csv'))
    else:
    
        # use glob to get all the csv files in the folder
        SRI_csv_files = glob.glob(os.path.join('SRI/','*.csv'))

        df_list = []

        # loop over the list of csv files
        for f in SRI_csv_files:
            dict = {}
            regex = re.compile(r'\d+')
            segment_id =  [int(x) for x in regex.findall(f)][0]
            dict['segment_id'] = segment_id
            dict['time'] = time_stamp
            # read csv file
            globals()['segment_id_'+str(segment_id) + '_df']=pd.read_csv(f)
            # Convert the time to datetime64
            globals()['segment_id_'+str(segment_id) + '_df']['time'] = pd.to_datetime(globals()['segment_id_'+str(segment_id) + '_df'].time)
            # Filter data for a specified time
            dict['SRI']=globals()['segment_id_'+str(segment_id) + '_df'].loc[globals()['segment_id_'+str(segment_id) + '_df']['time'] == time_stamp,'SRI'].iloc[0]
            
            data = pd.read_json(f"https://data.cityofchicago.org/resource/sxs8-h27x.json?$where=time%20between%20%27{'2019-09-27T07:00:00'}%27%20and%20%27{'2019-09-28T07:00:00'}%27&segment_id={segment_id}")
            dict['start_latitude']=data.iloc[0]['start_latitude']
            dict['start_longitude']=data.iloc[0]['start_longitude']
            dict['end_latitude']=data.iloc[0]['end_latitude']
            dict['end_longitude']=data.iloc[0]['end_longitude']
            globals()['filtered_'+'segment_id_'+str(segment_id) + '_df']=pd.DataFrame(dict,index=[segment_id])

            df_list.append(globals()['filtered_'+'segment_id_'+str(segment_id) + '_df'])
        globals()[time_stamp+'SRI_df'] = pd.concat(df_list) #merge data into one dataset
        ts = str(time_stamp)[:10]
        globals()[time_stamp+'SRI_df'].to_csv(Path('map/SRI_' + str(ts) + '.csv'))

In [3]:
SRI_orgnize('2019-09-28T07:00:00')
globals()['2019-09-28T07:00:00'+'SRI_df'].head()

,Unnamed: 0,segment_id,time,SRI,start_latitude,start_longitude,end_latitude,end_longitude
0,1000,1000,2019-09-28T07:00:00,1.919436,42.011599,-87.728599,42.011720,-87.709057
1,1001,1001,2019-09-28T07:00:00,1.789435,42.011720,-87.709057,42.011976,-87.699747
2,1003,1003,2019-09-28T07:00:00,1.591629,42.012163,-87.690242,42.012105,-87.683067
3,100,100,2019-09-28T07:00:00,1.083526,41.735789,-87.662882,41.743076,-87.663068
4,1011,1011,2019-09-28T07:00:00,-0.001779,42.011854,-87.806576,42.011596,-87.816326


In [4]:
SRI_orgnize('2019-09-21T07:00:00')
globals()['2019-09-21T07:00:00'+'SRI_df'].head()

,Unnamed: 0,segment_id,time,SRI,start_latitude,start_longitude,end_latitude,end_longitude
0,1000,1000,2019-09-21T07:00:00,1.919436,42.011599,-87.728599,42.011720,-87.709057
1,1001,1001,2019-09-21T07:00:00,1.789435,42.011720,-87.709057,42.011976,-87.699747
2,1003,1003,2019-09-21T07:00:00,1.677996,42.012163,-87.690242,42.012105,-87.683067
3,100,100,2019-09-21T07:00:00,-0.755735,41.735789,-87.662882,41.743076,-87.663068
4,1011,1011,2019-09-21T07:00:00,-0.001779,42.011854,-87.806576,42.011596,-87.816326


In [5]:
SRI_orgnize('2019-10-19T07:00:00')
globals()['2019-10-19T07:00:00'+'SRI_df'].head()

,Unnamed: 0,segment_id,time,SRI,start_latitude,start_longitude,end_latitude,end_longitude
0,1000,1000,2019-10-19T07:00:00,1.919436,42.011599,-87.728599,42.011720,-87.709057
1,1001,1001,2019-10-19T07:00:00,1.789435,42.011720,-87.709057,42.011976,-87.699747
2,1003,1003,2019-10-19T07:00:00,-0.151279,42.012163,-87.690242,42.012105,-87.683067
3,100,100,2019-10-19T07:00:00,0.923135,41.735789,-87.662882,41.743076,-87.663068
4,1011,1011,2019-10-19T07:00:00,-0.001779,42.011854,-87.806576,42.011596,-87.816326


In [6]:
def RRI_orgnize(time_stamp,station_id_list):
    df_list = []
    for station_id in station_id_list:
        dict = {}


        dict['station_id']= station_id
        dict['time'] = time_stamp
        globals()['station_id_'+str(station_id) + '_df']=pd.read_csv(Path('RRI/station_' + str(station_id) + '_RRI.csv'))
        dict['RRI']=globals()['station_id_'+str(station_id) + '_df'].loc[globals()['station_id_'+str(station_id) + '_df']['date'] == time_stamp,'RRI'].iloc[0]
        globals()['filtered_'+'station_id_'+str(station_id) + '_df']=pd.DataFrame(dict,index=[station_id])
        df_list.append(globals()['filtered_'+'station_id_'+str(station_id) + '_df'])

        
    globals()[time_stamp+'RRI_df'] = pd.concat(df_list) #merge data into one dataset
    globals()[time_stamp+'RRI_df'].to_csv(Path('map/RRI_' + str(time_stamp) + '.csv'))

In [7]:
RRI_orgnize('2019-09-28',[40890,40820])
globals()['2019-09-28'+'RRI_df'].head()

,station_id,time,RRI
40890,40890,2019-09-28,8.468513
40820,40820,2019-09-28,-4.262072


In [8]:
RRI_orgnize('2019-09-21',[40890,40820])
globals()['2019-09-21'+'RRI_df'].head()

,station_id,time,RRI
40890,40890,2019-09-21,3.327505
40820,40820,2019-09-21,3.894958


In [9]:
RRI_orgnize('2019-10-19',[40890,40820])
globals()['2019-10-19'+'RRI_df'].head()

,station_id,time,RRI
40890,40890,2019-10-19,2.008871
40820,40820,2019-10-19,2.811972


## Create a map centered on Chicago.

In [10]:
def SRI_gradient(middle,max,num):
    scale = 255 / (max-middle)

    if num > middle:
        return '#%02x%02x%02x' % (255 - int( ( num - middle ) * scale), 0, 0)
    else:
        return '#00FF00'

def RRI_gradient(middle,max,num):
    scale = (8-2) / (max-middle)

    if num > middle:
        return 2+scale*(num-middle)
    else:
        return 2


In [13]:
def visulization(time_stamp):    
    m = folium.Map(location=[41.8781, -87.6798], zoom_start=11,title='SRI on' + time_stamp)

    for index, row in globals()[time_stamp+'T07:00:00'+'SRI_df'].iterrows():
        start_lat = row['start_latitude']
        # print(start_lat)
        start_lon = row['start_longitude']
        end_lat = row['end_latitude']
        end_lon = row['end_longitude']
        SRI = row['SRI']
        points = []
        points.append([start_lat, start_lon])
        points.append([end_lat, end_lon])
        folium.PolyLine(points,color=SRI_gradient(2,10,SRI)).add_to(m)

    for index, row in globals()[time_stamp+'RRI_df'].iterrows():
        point = []
        if row['station_id'] == 40890:
            point.append([41.981127, -87.900876])
            RRI = row['RRI']
            folium.Marker((41.981127, -87.900876)).add_to(m)
            folium.CircleMarker((41.981127, -87.900876),radius=RRI_gradient(0,10,RRI),fill=False,color='black').add_to(m)
        else:
            point.append([41.983507, -87.859388])
            RRI = row['RRI']
            folium.Marker((41.983507, -87.859388)).add_to(m)
            folium.CircleMarker((41.983507, -87.859388),radius=RRI_gradient(0,10,RRI),fill=False,color='black').add_to(m)


    #create color legend
    colormap = cm.LinearColormap(colors=[(0,255,0),(255 - int( ( 2 - 2 ) * 255 / (10-2)), 0, 0), (255 - int( ( 10 - 2 ) * 255 / (10-2)), 0, 0)],
                             index=[0, 2, 10], vmin=2, vmax=10,
                             caption='Speed Reduction Index (SRI)')
    m.add_child(colormap)

    
    title_html = '''
                <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                '''.format(time_stamp)
    m.get_root().html.add_child(folium.Element(title_html))

    
    # Display the map.
    display(m)

    # Save map
    filepath = Path('map/station_' + str(time_stamp) + '.html')
    m.save(filepath)
    # mapUrl = 'file://{0}/{1}'.format(os.getcwd(), filepath)

    # # use selenium to save the html as png image
    # driver = webdriver.Firefox()
    # driver.get(mapUrl)

    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save(Path('map/station_' + str(time_stamp)+'.png'))

visulization('2019-09-28')
visulization('2019-09-21')
visulization('2019-10-19')


## Plot the corresponding SRI

In [16]:
def page_through_ridership_data(station_id,start_time,end_time):

    '''
    Name of Function: page_through_data
    Purpose of Function: to read the traffic data page by page
    Inputs: 
            -station_id
            -start_time
            -end_time
    Expected Outputs: 
            -station_id_xxxx_df
    ''' 

    # The $offset is the number of records into a dataset that you want to start, indexed at 0.
    offset  = 0 # start with no offset
    # segment_id = i #choose a road segment
    df_list = []
    data = pd.read_json(f"https://data.cityofchicago.org/resource/5neh-572f.json?$where=date%20between%20%27{start_time}%27%20and%20%27{end_time}%27&station_id={station_id}&$limit=50000&$offset={offset}&$order=date")
    #Notice the speed!=-1 condition specified in the url, it can help get rid of empty data
    while len(data)==50000: #check num of rows in each page, if num of rows == maximum value for $limit, next page exist
        df_list.append(data) #append data to the list
        offset = offset + 50000 #move to next page
        data = pd.read_json(f"https://data.cityofchicago.org/resource/sxs8-h27x.json?$where=date%20between%20%27{start_time}%27%20and%20%27{end_time}%27&station_id={station_id}&$limit=50000&$offset={offset}&$order=date")
    df_list.append(data)
    globals()['station_id_'+str(station_id) + '_ridership'] = pd.concat(df_list) #merge data into one dataset

In [129]:
station_id = 40890
page_through_ridership_data(station_id,'2018-03-05T00:00:00','2021-03-05T00:00:00')
ridership_before = globals()['station_id_'+str(station_id) + '_ridership'].loc[globals()['station_id_'+str(station_id) + '_ridership']['date'] == '2019-09-21']
ridership_during = globals()['station_id_'+str(station_id) + '_ridership'].loc[globals()['station_id_'+str(station_id) + '_ridership']['date'] == '2019-09-28']
ridership_after = globals()['station_id_'+str(station_id) + '_ridership'].loc[globals()['station_id_'+str(station_id) + '_ridership']['date'] == '2019-10-19']
fig = make_subplots(rows=2, cols=1,subplot_titles=["Ridership", "Ridership Reduction Index (RRI)"],shared_xaxes=True,)

fig.add_trace(go.Scatter(x=globals()['station_id_'+str(station_id) + '_ridership'].date, y=globals()['station_id_'+str(station_id) + '_ridership'].rides, mode='lines',),row=1, col=1,)
fig.add_trace(go.Scatter(x=ridership_before.date, y=ridership_before.rides, mode='markers+text',marker=dict(color='black',size=5),text='2019-09-21:' + str(ridership_before.iloc[0]['rides']),textposition="bottom center"),row=1, col=1,)
fig.add_trace(go.Scatter(x=ridership_during.date, y=ridership_during.rides, mode='markers+text',marker=dict(color='black',size=5),text='2019-09-28:' + str(ridership_during.iloc[0]['rides']),textposition="bottom center"),row=1, col=1,)
fig.add_trace(go.Scatter(x=ridership_after.date, y=ridership_after.rides, mode='markers+text',marker=dict(color='black',size=5),text='2019-10-19:' + str(ridership_after.iloc[0]['rides']),textposition="bottom center"),row=1, col=1,)


RRI_before =globals()['station_id_'+str(station_id) + '_df'].loc[globals()['station_id_'+str(station_id) + '_df']['date'] == '2019-09-21']
RRI_during = globals()['station_id_'+str(station_id) + '_df'].loc[globals()['station_id_'+str(station_id) + '_df']['date'] == '2019-09-28']
RRI_after = globals()['station_id_'+str(station_id) + '_df'].loc[globals()['station_id_'+str(station_id) + '_df']['date'] == '2019-10-19']
fig.add_trace(go.Scatter(x=globals()['station_id_'+str(station_id) + '_df'].date, y=globals()['station_id_'+str(station_id) + '_df'].RRI, mode='lines',),row=2, col=1,)
fig.add_trace(go.Scatter(x=RRI_before.date, y=RRI_before.RRI, mode='markers+text',marker=dict(color='black',size=5),text='2019-09-21:' + "{:.2f}".format(RRI_before.iloc[0]['RRI']),textposition="bottom center"),row=2, col=1,)
fig.add_trace(go.Scatter(x=RRI_during.date, y=RRI_during.RRI, mode='markers+text',marker=dict(color='black',size=5),text='2019-09-28:' + "{:.2f}".format(RRI_during.iloc[0]['RRI']),textposition="bottom center"),row=2, col=1,)
fig.add_trace(go.Scatter(x=RRI_after.date, y=RRI_after.RRI, mode='markers+text',marker=dict(color='black',size=5),text='2019-10-19:' + "{:.2f}".format(RRI_after.iloc[0]['RRI']),textposition="bottom center"),row=2, col=1,)

fig.update_layout(height=900, margin=dict(t=100),showlegend=False,title_text= str(globals()['station_id_'+str(station_id) + '_ridership'].iloc[0]['stationname']))
fig.update_xaxes(type="date", range=['2019-09-01', '2019-11-01'], row=2, col=1)
fig.show()